<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [4]</a>'.</span>

<center>
    <h1>C++ kernel based on xeus</h1>
</center>

A Jupyter kernel for C++ based on the `CppInterOp`, a clang based C++ Interoperability Library and the `xeus` native implementation of the Jupyter protocol, xeus.

- GitHub repository: https://github.com/compiler-research/xeus-cpp
- Documentation: https://xeus-cpp.readthedocs.io/en/latest/

## Usage

<div style="background: #efffed;
            border: 1px solid grey;
            margin: 8px 0 8px 0;
            text-align: center;
            padding: 8px; ">
    <i class="fa-play fa" 
       style="font-size: 40px;
              line-height: 40px;
              margin: 8px;
              color: #444;">
    </i>
    <div>
    To run the selected code cell, hit <pre style="background: #efffed">Shift + Enter</pre>
    </div>
</div>

## Output and error streams

`std::cout` and `std::cerr` are redirected to the notebook frontend.

In [1]:
#include <iostream>

std::cout << "some output" << std::endl;

some output


In [2]:
std::cerr << "some error" << std::endl;

In [3]:
#include <stdexcept>

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [4]:
throw std::runtime_error("Unknown exception");

Standard Exception: : Unknown exception

In [ ]:
int j = 5;
std::cout << j << std::endl;

# Interpreting the C++ programming language

You can define functions, classes, templates, etc ...

## Functions

In [ ]:
double sqr(double a)
{
    return a * a;
}

In [ ]:
double a = 2.5;
double asqr = sqr(a);
std::cout << asqr << std::endl;

## Classes

In [ ]:
class Foo
{
public:

    virtual ~Foo() {}
    
    virtual void print(double value) const
    {
        std::cout << "Foo value = " << value << std::endl;
    }
};

In [ ]:
Foo bar;
bar.print(1.2);

## Polymorphism

In [ ]:
class Bar : public Foo
{
public:

    virtual ~Bar() {}
    
    virtual void print(double value) const
    {
        std::cout << "Bar value = " << 2 * value << std::endl;
    }
};

In [ ]:
Foo* bar2 = new Bar;
bar2->print(1.2);
delete bar2;

## Templates

In [ ]:
#include <typeinfo>

template <class T>
class FooT
{
public:
    
    explicit FooT(const T& t) : m_t(t) {}
    
    void print() const
    {
        std::cout << typeid(T).name() << " m_t = " << m_t << std::endl;
    }
    
private:
    
    T m_t;
};

In [ ]:
FooT<double> foot(1.2);
foot.print();

## Documentation

 - Documentation for types of the standard library is retrieved on cppreference.com.

In [ ]:
?std::vector

## Using the `display_data` mechanism

For a user-defined type `T`, the rich rendering in the notebook and JupyterLab can be enabled by by implementing the function `nl::json mime_bundle_repr(const T& im)`, which returns the JSON mime bundle for that type.

### Image example

In [ ]:
#include <string>
#include <fstream>

#include "nlohmann/json.hpp"

#include "xeus/xbase64.hpp"

namespace nl = nlohmann;

namespace im
{
    struct image
    {   
        inline image(const std::string& filename)
        {
            std::ifstream fin(filename, std::ios::binary);   
            m_buffer << fin.rdbuf();
        }
        
        std::stringstream m_buffer;
    };
    
    nl::json mime_bundle_repr(const image& i)
    {
        auto bundle = nl::json::object();
        bundle["image/png"] = xeus::base64encode(i.m_buffer.str());
        return bundle;
    }
}

In [ ]:
im::image marie("images/marie.png");

In [ ]:
#include "xcpp/xdisplay.hpp"

In [ ]:
xcpp::display(marie);

### Audio example

In [ ]:
#include <string>
#include <fstream>

#include "nlohmann/json.hpp"

#include "xeus/xbase64.hpp"

namespace nl = nlohmann;

namespace au
{
    struct audio
    {   
        inline audio(const std::string& filename)
        {
            std::ifstream fin(filename, std::ios::binary);   
            m_buffer << fin.rdbuf();
        }
        
        std::stringstream m_buffer;
    };
    
    nl::json mime_bundle_repr(const audio& a)
    {
        auto bundle = nl::json::object();
        bundle["text/html"] =
           std::string("<audio controls=\"controls\"><source src=\"data:audio/wav;base64,")
           + xeus::base64encode(a.m_buffer.str()) +
            "\" type=\"audio/wav\" /></audio>";
        return bundle;
    }
}

In [ ]:
au::audio drums("audio/audio.wav");

In [ ]:
xcpp::display(drums);

### Update-display

In [ ]:
#include <string>
#include "xcpp/xdisplay.hpp"

#include "nlohmann/json.hpp"

namespace nl = nlohmann;

namespace ht
{
    struct html
    {   
        inline html(const std::string& content)
        {
            m_content = content;
        }
        std::string m_content;
    };

    nl::json mime_bundle_repr(const html& a)
    {
        auto bundle = nl::json::object();
        bundle["text/html"] = a.m_content;
        return bundle;
    }
}

// A blue rectangle
ht::html rect(R"(
<div style='
    width: 90px;
    height: 50px;
    line-height: 50px;
    background-color: blue;
    color: white;
    text-align: center;'>
Original
</div>)");

In [ ]:
xcpp::display(rect, "some_display_id");

In [ ]:
// Update the rectangle to be red
rect.m_content = R"(
<div style='
    width: 90px;
    height: 50px;
    line-height: 50px;
    background-color: red;
    color: white;
    text-align: center;'>
Updated
</div>)";

xcpp::display(rect, "some_display_id", true);

### Clear output

In [ ]:
#include <chrono>
#include <iostream>
#include <thread>

#include "xcpp/xdisplay.hpp"

In [ ]:
std::cout << "hello" << std::endl;
std::this_thread::sleep_for(std::chrono::seconds(1));
xcpp::clear_output();  // will flicker when replacing "hello" with "goodbye"
std::this_thread::sleep_for(std::chrono::seconds(1));
std::cout << "goodbye" << std::endl;

In [ ]:
std::cout << "hello" << std::endl;
std::this_thread::sleep_for(std::chrono::seconds(1));
xcpp::clear_output(true);  // prevents flickering
std::this_thread::sleep_for(std::chrono::seconds(1));
std::cout << "goodbye" << std::endl;

# Taking input from the user

In [ ]:
std::string name;
std::cin >> name;

In [ ]:
std::cout << "Your name is" << name;